In [20]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Define the RBF Network class
class RBFNetwork:
    def __init__(self, n_rbf_units, n_input, n_output, lr):
        self.n = n_rbf_units  # Number of RBF units
        self.n_input = n_input  # Number of input features
        self.n_output = n_output  # Number of output features
        self.lr = lr  # Learning rate
        self.weights = np.random.randn(self.n, self.n_output) * 0.01  # Random initialization of weights
        self.bias = np.zeros((1, self.n_output))  # Initialize bias
        self.centers = None  # RBF centers
        self.std = None  # RBF standard deviations

    def fit(self, X, k):
        # Perform KMeans clustering to find centers for RBF units
        kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
        self.centers = kmeans.cluster_centers_
        self.std = np.array([np.std(X[kmeans.labels_ == i]) for i in range(k)])  # Calculate std deviations

    def rbf(self, x, center, variance):
        # Gaussian function (RBF)
        return np.exp(-np.linalg.norm(x - center) ** 2 / (2 * variance ** 2))

    def _calculate_activations(self, X):
        G = np.zeros((X.shape[0], self.n))
        for i, x in enumerate(X):
            for j, center in enumerate(self.centers):
                G[i, j] = self.rbf(x, center, self.std[j])
        return G

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            G = self._calculate_activations(X)
            y_pred = G.dot(self.weights) + self.bias
            error = y - y_pred
            
            # Gradient descent
            gradient = -G.T.dot(error) / len(X)
            self.weights -= self.lr * gradient
            self.bias -= self.lr * np.mean(error, axis=0)

    def predict(self, X):
        G = self._calculate_activations(X)
        F_x = G.dot(self.weights) + self.bias
        return F_x

# Generate sinusoidal data with noise
def generate_data(num_samples=100):
    X = np.linspace(0, 2 * np.pi, num_samples).reshape(-1, 1)  # Input values
    y = np.sin(X) + np.random.normal(0, 0.1, X.shape)  # Sinusoidal function with noise
    return X, y

# Main execution
if __name__ == "__main__":
    # Generate data
    X, y = generate_data(num_samples=100)

    # Train the RBF network
    rbf_net = RBFNetwork(n_rbf_units=10, n_input=1, n_output=1, lr=0.01)  # Specify n_output
    rbf_net.fit(X, k=10)
    rbf_net.train(X, y, epochs=100)

    # Make predictions
    y_pred = rbf_net.predict(X)

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.scatter(X, y, label='Noisy Data', color='blue')
    plt.plot(X, y_pred, label='RBF Network Prediction', color='red', linewidth=2)
    plt.title('RBF Network Fitting to Noisy Sinusoidal Data')
    plt.xlabel('X')
    plt.ylabel('y')
    plt.legend()
    plt.grid()
    plt.show()


ModuleNotFoundError: No module named 'matplotlib'